In [35]:
import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
torch.__version__

'2.1.1+cpu'

In [37]:
class BinaryClassification(nn.Module):
  def __init__(self, embed_size, device):
    super(BinaryClassification, self).__init__()
    # Number of input features is embed_size.
    self.layer_1 = nn.Linear(embed_size, 64)
    self.layer_2 = nn.Linear(64, 64)
    self.layer_out = nn.Linear(64, 1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.1)
    self.batchnorm1 = nn.BatchNorm1d(64)
    self.batchnorm2 = nn.BatchNorm1d(64)
    self.device = device

  def forward(self, inputs):
    # print("start binary classification")
    # print(inputs.shape)
    # print(inputs)
    x = self.relu(self.layer_1(inputs))
    x = self.batchnorm1(x)
    x = self.relu(self.layer_2(x))
    x = self.batchnorm2(x)
    x = self.dropout(x)
    x = self.layer_out(x)
    #if math.isnan (x[0][0]):
    #  print(src)

    return x

In [38]:
# class Classifier(nn.Module):
#   def __init__(self, d_model, seq_len, nhead, dim_feedforward, nlayers, device, dropout = 0.5):
#     super(Classifier, self).__init__()
#     self.d_model = d_model
#     self.seq_len = seq_len
#     self.nhead = nhead
#     self.dim_feedforward = dim_feedforward
#     self.nlayers = nlayers
#     self.device = device
#     #self.pos_encoder = PositionalEncoding(d_model, dropout)
#     self.position_embedding = nn.Embedding(seq_len, d_model)
#     encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
#     self.encoder = TransformerEncoder(encoder_layer, nlayers)
#     self.binary_classifier = BinaryClassification(seq_len*d_model, device)


#   def forward(self, src: Tensor) -> Tensor:
#     # print("Classifier forwrd")
#     """
#     Args:
#         src: Tensor, shape [seq_len, batch_size]
#         src_mask: Tensor, shape [seq_len, seq_len]
#     Returns:
#         output Tensor of shape [seq_len, batch_size, ntoken]
#     """
#     N, seq_length, embed_size = src.shape
#     positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

#     # print(f"src before positional embeddings: {src.shape}")
#     src_ = src + self.position_embedding(positions)
#     # print(f"src after positional embeddings: {src_.shape}")
#     # print(f"src after positional embeddings: {src.shape}")
#     # print(src_)
#     # print("before encoder")
#     output = self.encoder(src_)
#     # print(f"encoder output shape: {output.shape}")
#     # print(output[0])
#     # print(output)
#     # print("after encoder")
#     output = self.binary_classifier(torch.reshape(output, (N, seq_length*embed_size))) 
#     # output_tx_f = torch.reshape(output, (N, seq_length*embed_size))
#     # output = self.binary_classifier(torch.cat((output_tx_f), dim=1)) ##
#     return output

In [39]:
class Classifier(nn.Module):
  def __init__(self, d_model, seq_len, nhead, dim_feedforward, nlayers, device, dropout = 0.5):
    super(Classifier, self).__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.nhead = nhead
    self.dim_feedforward = dim_feedforward
    self.nlayers = nlayers
    self.device = device
    
    self.position_embedding = nn.Embedding(seq_len, d_model)
    encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
    self.encoder = TransformerEncoder(encoder_layer, nlayers)
    self.binary_classifier = BinaryClassification(seq_len*d_model, device)

  def forward(self, src: Tensor) -> Tensor:

    """
    Args:
        src: Tensor, shape [seq_len, batch_size]
        src_mask: Tensor, shape [seq_len, seq_len]
    Returns:
        output Tensor of shape [seq_len, batch_size, ntoken]
    """
    N, seq_length, embed_size = src.shape
    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
    src_ = src + self.position_embedding(positions)
    output1 = self.encoder(src_)
    # print(output1.shape)
    # print(output1)
    output = self.binary_classifier(torch.reshape(output1, (N, seq_length*embed_size))) 

    return output, output1

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#hyperparameters
dim_feedforward=16
#dim_feedforward=32
nlayers_tx=8

In [41]:
model = Classifier(d_model=11, seq_len=108, nhead=11, dim_feedforward=8, nlayers=8, device=device)
model.to(device)

c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Classifier(
  (position_embedding): Embedding(108, 11)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=11, out_features=11, bias=True)
        )
        (linear1): Linear(in_features=11, out_features=8, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=8, out_features=11, bias=True)
        (norm1): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (binary_classifier): BinaryClassification(
    (layer_1): Linear(in_features=1188, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=1, bias=True)
    

In [42]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    # print("y_pred",torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [43]:
x = np.load("data\other\data.npy")
y = np.load("data\other\labels.npy")

In [44]:
print(x.shape)
print(y.shape)

(301, 108, 11)
(301,)


In [45]:
print(x[:5])

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   2.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[6.25000000e-01 6.25000000e-01 1.08000000e+20 ... 8.00000000e+00
   2.00000000e+00 1.00000000e+01]
  [1.00000000e+00 1.00000000e+00 1.00000000e+19 ... 9.00000000e+00
   1.00000000e+00 1.00000000e+01]
  [5.71428571e-01 5.71428571e-01 1.90000000e+19 ... 7.00000000e+00
   1.00000000e+00 1.00000000e+01]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.00000000e+00
   0.00000

In [46]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming you have loaded your data into a variable called data_array
# data_array should be of shape (301, 108, 11)

# Reshape the array to (301*108, 11) for normalization
reshaped_data = x.reshape((-1, 11))

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the reshaped data and transform it
normalized_data = scaler.fit_transform(reshaped_data)

# Reshape the normalized data back to the original shape
normalized_data_array = normalized_data.reshape(x.shape)

# Now, normalized_data_array contains the normalized data with shape (301, 108, 11)
print(normalized_data_array[:5])

[[[-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
    6.17300954e+00 -2.28408205e-01]
  [-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
    2.93045142e+00 -2.28408205e-01]
  [-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
    2.93045142e+00 -2.28408205e-01]
  ...
  [-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
   -3.12106692e-01 -2.28408205e-01]
  [-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
   -3.12106692e-01 -2.28408205e-01]
  [-1.02945209e-01 -1.14356860e-01 -1.38146329e-02 ... -2.37866691e-01
   -3.12106692e-01 -2.28408205e-01]]

 [[ 8.86766055e+00  8.05507759e+00 -4.84967205e-03 ...  6.99221315e+00
    6.17300954e+00  4.61193986e+00]
  [ 1.42500240e+01  1.29567383e+01 -1.29845439e-02 ...  7.89597313e+00
    2.93045142e+00  4.61193986e+00]
  [ 8.09875148e+00  7.35484035e+00 -1.22374638e-02 ...  6.08845317e+00
    2.93045142e+00  4.61193986e+00]
  ...
  [-1.02945209e-01 -1.1

In [47]:
X, X_test, Y, y_test = train_test_split(normalized_data_array, y, test_size=0.1, random_state=42, stratify=y) #test split

In [48]:

# Assuming you have split the data using the code provided in the previous response

# Check class distribution in original 'y'
unique_classes_y, counts_y = np.unique(y, return_counts=True)
print("Original class distribution:")
print(dict(zip(unique_classes_y, counts_y)))

# Check class distribution in 'y_train'
unique_classes_train, counts_train = np.unique(Y, return_counts=True)
print("\nClass distribution in y_train:")
print(dict(zip(unique_classes_train, counts_train)))

# Check class distribution in 'y_test'
unique_classes_test, counts_test = np.unique(y_test, return_counts=True)
print("\nClass distribution in y_test:")
print(dict(zip(unique_classes_test, counts_test)))


Original class distribution:
{0: 204, 1: 97}

Class distribution in y_train:
{0: 183, 1: 87}

Class distribution in y_test:
{0: 21, 1: 10}


In [49]:
np.save("X_test.npy",X_test)
np.save("y_test.npy",y_test)

In [50]:
#load test
X_test = np.load("X_test.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

In [51]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y) #validation split

In [52]:
np.save("X_train.npy",X_train)
np.save("y_train.npy",y_train)
np.save("X_val.npy",X_val)
np.save("y_val.npy",y_val)

In [53]:
#load train val
X_train = np.load("X_train.npy", allow_pickle=True)
y_train = np.load("y_train.npy", allow_pickle=True)
X_val = np.load("X_val.npy", allow_pickle=True)
y_val = np.load("y_val.npy", allow_pickle=True)

In [54]:
print(X_train.shape)

(243, 108, 11)


In [55]:
X_train = X_train[:19656]

In [56]:
print(X_val.shape)

(27, 108, 11)


In [57]:
X_val = X_val[:2184]

In [58]:
print(X_test.shape)

(31, 108, 11)


In [59]:
print(np.sum(y_train)/y_train.shape[0])
print(np.sum(y_val)/y_val.shape[0])
print(np.sum(y_test)/y_test.shape[0])

0.32098765432098764
0.3333333333333333
0.3225806451612903


In [60]:
from torch.utils.data import Dataset, DataLoader

In [61]:
## train data
from sklearn import preprocessing
class TrainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(X_train, y_train)

## val data
class ValData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


val_data = ValData(X_val, y_val)

## test data
class TestData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


test_data = TestData(X_test, y_test)

In [62]:
EPOCHS = 50
BATCH_SIZE = 8
LEARNING_RATE = 0.001

In [63]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [64]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [65]:
print(len(train_loader))
print(train_loader)

31


Hyperparameter tuning

In [68]:
import itertools

# Define the hyperparameter values to test
hyperparameters = {
    'epochs': [10, 20],
    'batch_size': [8, 16],
    'learning_rate': [0.001, 0.01]
}

# Generate all combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Iterate over each combination
for params in param_combinations:
    # Unpack the parameters
    epochs, batch_size, learning_rate = params

    # Create a new model and optimizer with the current hyperparameters
    model = Classifier(d_model=11, seq_len=108, nhead=11, dim_feedforward=8, nlayers=8, device=device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Training loop
    for e in range(1, epochs + 1):
        # ... (your existing training code)
        epoch_loss = 0
        epoch_acc = 0
        model.train()
        for X_batch, y_batch in train_loader:
            #print("w.requires_grad:",X_batch.requires_grad)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            y_pred,result_before_bin_classifier = model(X_batch.float())
            # print(f"y_pred = {y_pred}")
            #y_pred = (y_pred>0.5).float()
            #print(f"y_pred = {y_pred}")
            #print(f"y_batch.unsqueeze(1) = {y_batch.unsqueeze(1)}")

            loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
            acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())


            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        val_loss = 0
        val_accuracy = 0
        model.eval()
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred,result_before_bin_classifier = model(X_batch.float())
            loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
            acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())
            val_loss += loss.item()
            val_accuracy += acc.item()

        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f} | Val_Loss: {val_loss/len(test_loader):.5f} | Val_Acc: {val_accuracy/len(test_loader):.3f}')

    # Print the results for each hyperparameter combination
    print(f'Hyperparameters: Epochs={epochs}, Batch Size={batch_size}, Learning Rate={learning_rate}')
    print(f'Final Training Loss: {epoch_loss/len(train_loader):.5f}, Final Training Acc: {epoch_acc/len(train_loader):.3f}')
    print(f'Final Validation Loss: {val_loss/len(test_loader):.5f}, Final Validation Acc: {val_accuracy/len(test_loader):.3f}')
    print('\n')


c:\Users\sandu\Desktop\sem7\fyp\App\.venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 001: | Loss: 0.72948 | Acc: 51.290 | Val_Loss: 0.65223 | Val_Acc: 58.065
Epoch 002: | Loss: 0.67789 | Acc: 58.065 | Val_Loss: 0.60712 | Val_Acc: 54.839
Epoch 003: | Loss: 0.65655 | Acc: 58.645 | Val_Loss: 0.54251 | Val_Acc: 77.419
Epoch 004: | Loss: 0.67978 | Acc: 62.194 | Val_Loss: 0.52271 | Val_Acc: 80.645
Epoch 005: | Loss: 0.62838 | Acc: 68.935 | Val_Loss: 0.57614 | Val_Acc: 74.194
Epoch 006: | Loss: 0.59938 | Acc: 70.097 | Val_Loss: 0.53984 | Val_Acc: 80.645
Epoch 007: | Loss: 0.59970 | Acc: 70.839 | Val_Loss: 0.55920 | Val_Acc: 77.419
Epoch 008: | Loss: 0.58867 | Acc: 71.903 | Val_Loss: 0.63738 | Val_Acc: 74.194
Epoch 009: | Loss: 0.61617 | Acc: 69.484 | Val_Loss: 0.56246 | Val_Acc: 77.419
Epoch 010: | Loss: 0.62871 | Acc: 65.935 | Val_Loss: 0.53363 | Val_Acc: 77.419
Hyperparameters: Epochs=10, Batch Size=8, Learning Rate=0.001
Final Training Loss: 0.62871, Final Training Acc: 65.935
Final Validation Loss: 0.53363, Final Validation Acc: 77.419


Epoch 001: | Loss: 0.75554 |

In [66]:

for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        #print("w.requires_grad:",X_batch.requires_grad)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred,result_before_bin_classifier = model(X_batch.float())
        # print(f"y_pred = {y_pred}")
        #y_pred = (y_pred>0.5).float()
        #print(f"y_pred = {y_pred}")
        #print(f"y_batch.unsqueeze(1) = {y_batch.unsqueeze(1)}")

        loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
        acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())


        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    val_loss = 0
    val_accuracy = 0
    model.eval()
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred,result_before_bin_classifier = model(X_batch.float())
        loss = criterion(y_pred.float(), y_batch.unsqueeze(1).float())
        acc = binary_acc(y_pred.float(), y_batch.unsqueeze(1).float())
        val_loss += loss.item()
        val_accuracy += acc.item()

    # Check for early stopping
        # if val_accuracy > best_validation_accuracy:
        #     best_validation_accuracy = val_accuracy
        #     no_improvement_counter = 0
        #     # Save the trained best fakebert if needed
        #     torch.save(fakebert.state_dict(), '/content/drive/Shareddrives/test/FYP/fake-news/fakebert-twitterus.pth')
        # else:
        #     no_improvement_counter += 1

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f} | Val_Loss: {val_loss/len(test_loader):.5f} | Val_Acc: {val_accuracy/len(test_loader):.3f}')
    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.75290 | Acc: 47.323 | Val_Loss: 0.83656 | Val_Acc: 41.935
Epoch 002: | Loss: 0.69539 | Acc: 54.065 | Val_Loss: 0.74423 | Val_Acc: 64.516
Epoch 003: | Loss: 0.63829 | Acc: 60.645 | Val_Loss: 0.66293 | Val_Acc: 74.194
Epoch 004: | Loss: 0.67381 | Acc: 60.032 | Val_Loss: 1.32817 | Val_Acc: 64.516
Epoch 005: | Loss: 0.63716 | Acc: 66.290 | Val_Loss: 3.41810 | Val_Acc: 54.839
Epoch 006: | Loss: 0.63916 | Acc: 64.581 | Val_Loss: 0.61135 | Val_Acc: 70.968
Epoch 007: | Loss: 0.62134 | Acc: 69.774 | Val_Loss: 0.63267 | Val_Acc: 64.516
Epoch 008: | Loss: 0.62057 | Acc: 69.226 | Val_Loss: 0.72468 | Val_Acc: 64.516
Epoch 009: | Loss: 0.62982 | Acc: 66.097 | Val_Loss: 1.03987 | Val_Acc: 64.516
Epoch 010: | Loss: 0.63087 | Acc: 65.000 | Val_Loss: 2.33139 | Val_Acc: 77.419
Epoch 011: | Loss: 0.63943 | Acc: 64.968 | Val_Loss: 8.73172 | Val_Acc: 77.419
Epoch 012: | Loss: 0.61811 | Acc: 68.742 | Val_Loss: 23.38268 | Val_Acc: 67.742
Epoch 013: | Loss: 0.60736 | Acc: 69.161 | Val_Loss

In [34]:
#PATH = './DeFi_Supervised_v1.pth'
#PATH = './DeFi_Ablation_v2.pth'
PATH = './PonziShield_tr_v1.pth'
torch.save(model, PATH)




RuntimeError: 
Return value was annotated as having type Tensor but is actually of type Tuple[Tensor, Tensor]:
  File "C:\Users\sandu\AppData\Local\Temp\ipykernel_22456\1340755348.py", line 33
    output = self.binary_classifier(torch.reshape(output1, (N, seq_length*embed_size))) 
  
    return output, output1
    ~~~~~~~~~~~~~~~~~~~~~~ <--- HERE


In [ ]:
#PATH = './DeFi_Ablation_v1.pth'
#PATH = './DeFi_Ablation_v2.pth'
PATH = './PonziShield_tr_v1.pth'
loaded_model = torch.load(PATH)
loaded_model.eval()

Classifier(
  (position_embedding): Embedding(108, 11)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=11, out_features=11, bias=True)
        )
        (linear1): Linear(in_features=11, out_features=8, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=8, out_features=11, bias=True)
        (norm1): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((11,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (binary_classifier): BinaryClassification(
    (layer_1): Linear(in_features=1188, out_features=64, bias=True)
    (layer_2): Linear(in_features=64, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=1, bias=True)
    

In [99]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# import seaborn as sn
import pandas as pd

In [100]:
y_pred = []
y_true = []
sigmoid = nn.Sigmoid()

for X_batch, y_batch in test_loader:
        #print("w.requires_grad:",X_batch.requires_grad)
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        results,result_before_bin_classifier = loaded_model(X_batch.float())
        y_batch_pred = torch.round(sigmoid(results))
        y_pred.extend(y_batch_pred.cpu().detach().numpy())
        y_true.extend(y_batch.cpu().detach().numpy())

In [101]:
count_true = 0
for i in range(len(y_pred)):
  if y_true[i]==y_pred[i]:
    count_true+=1
acc = count_true/len(y_pred)
print(len(y_test))
print(acc)

31
0.7741935483870968


In [102]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

(17, 4, 3, 7)

In [103]:
# Calculate Recall
recall = tp / (tp + fn)

# Calculate Precision
precision = tp / (tp + fp)

# Calculate F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))

Recall: 0.7000
Precision: 0.6364
F1 Score: 0.6667


In [40]:
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1), index = ['0','1'],
                     columns = ['0','1'])
plt.figure(figsize = (12,7))
# sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

<Figure size 1200x700 with 0 Axes>

**Two token**

In [ ]:
#Anubis
ankh_x1 = np.load("ankh_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0071]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.2284]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9375]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#full

#for i in range(50,108):
#  ankh_x1[i] = [0,0,0,0,0,0,0]

print(ankh_x1.shape)
ankh_x1 = np.reshape(ankh_x1, (1, 108, 7))
print(ankh_x1.shape)

ankh_x1 = torch.from_numpy(ankh_x1)
ankh_x1 = ankh_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(ankh_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[7.5255e-07]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Sushi
sushi_x1 = np.load("sushi_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0580]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9919]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  sushi_x1[i] = [0,0,0,0,0,0,0]

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9957]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Full

print(sushi_x1.shape)
sushi_x1 = np.reshape(sushi_x1, (1, 108, 7))
print(sushi_x1.shape)

sushi_x1 = torch.from_numpy(sushi_x1)
sushi_x1 = sushi_x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(sushi_x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[1.]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

**APE**

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving ape_full_x1.npy to ape_full_x1.npy


In [ ]:
#Ape
x1 = np.load("ape_full_x1.npy", allow_pickle=True)

In [ ]:
#500

for i in range(50,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.0006]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#750

for i in range(75,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9763]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#850

for i in range(85,108):
  x1[i] = [0,0,0,0,0,0,0]

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[0.9997]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [ ]:
#Full

print(x1.shape)
x1 = np.reshape(x1, (1, 108, 7))
print(x1.shape)

x1 = torch.from_numpy(x1)
x1 = x1.to(device)

sigmoid = nn.Sigmoid()
val = sigmoid(loaded_model(x1.float()))
print(val)
torch.round(val)

(108, 7)
(1, 108, 7)
tensor([[8.2050e-08]], device='cuda:0', grad_fn=<SigmoidBackward0>)


tensor([[0.]], device='cuda:0', grad_fn=<RoundBackward0>)